# Wise Transfer Funnel Analysis (v3): Performance Deep Dive

## 1. Setup and Data Preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

warnings.filterwarnings('ignore')

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
palette = sns.color_palette('Greens_r', n_colors=10)
sns.set_palette(palette)
IMAGE_DIR = 'images'
os.makedirs(IMAGE_DIR, exist_ok=True)

# Load and prepare the data
df = pd.read_csv('src/wise_funnel_events.csv')
df['dt'] = pd.to_datetime(df['dt'])
df['week'] = df['dt'].dt.isocalendar().week

print('Data loaded and prepared.')

## 2. Overall Funnel Health: New vs. Existing Users

In [ ]:
def analyze_funnel_by_experience(df_exp, experience_type):
    created = df_exp[df_exp['event_name'] == 'Transfer Created']['user_id'].nunique()
    funded = df_exp[df_exp['event_name'] == 'Transfer Funded']['user_id'].nunique()
    transferred = df_exp[df_exp['event_name'] == 'Transfer Transferred']['user_id'].nunique()
    return {'Created': created, 'Funded': funded, 'Transferred': transferred}

new_funnel_data = analyze_funnel_by_experience(df[df['experience'] == 'New'], 'New')
existing_funnel_data = analyze_funnel_by_experience(df[df['experience'] == 'Existing'], 'Existing')
funnel_comparison = pd.DataFrame({'New': new_funnel_data, 'Existing': existing_funnel_data})
new_conv = (funnel_comparison['New'] / funnel_comparison['New']['Created']) * 100
existing_conv = (funnel_comparison['Existing'] / funnel_comparison['Existing']['Created']) * 100
conv_comparison = pd.DataFrame({'New (%)': new_conv, 'Existing (%)': existing_conv})

fig, ax = plt.subplots(figsize=(12, 7))
conv_comparison.T.plot(kind='bar', ax=ax)
ax.set_title('New vs. Existing User Conversion Rates', fontsize=16, fontweight='bold')
ax.set_ylabel('Conversion Rate (%)')
ax.tick_params(axis='x', rotation=0)
plt.savefig(f'{IMAGE_DIR}/new_vs_existing_conversion.png')
plt.show()

![New vs. Existing User Conversion Rates](images/new_vs_existing_conversion.png)

## 3. Android Weekly Performance Analysis

In [ ]:
android_df = df[(df['experience'] == 'New') & (df['platform'] == 'Android')]
weekly_funnel = android_df.groupby('week')['event_name'].value_counts().unstack().fillna(0)
weekly_funnel = weekly_funnel.reindex(columns=['Transfer Created', 'Transfer Funded', 'Transfer Transferred']).fillna(0)
weekly_conv = weekly_funnel.div(weekly_funnel['Transfer Created'], axis=0).drop(columns=['Transfer Created']) * 100

fig, ax = plt.subplots(figsize=(14, 7))
weekly_conv.plot(kind='bar', ax=ax, width=0.8)
ax.set_title('Android Weekly Conversion Funnel for New Users', fontsize=16, fontweight='bold')
ax.set_ylabel('Conversion Rate (%)')
ax.set_xlabel('Week of the Year')
ax.tick_params(axis='x', rotation=45)
ax.legend(['Funded Rate', 'Transferred Rate'])
plt.tight_layout()
plt.savefig(f'{IMAGE_DIR}/android_weekly_conversion.png')
plt.show()

![Android Weekly Conversion](images/android_weekly_conversion.png)

## 4. Settled vs. Unsettled User Analysis

In [ ]:
new_user_ids = df[df['experience'] == 'New']['user_id'].unique()
all_new_users = df[df['user_id'].isin(new_user_ids)].drop_duplicates(subset='user_id').set_index('user_id')
transferred_users = df[df['event_name'] == 'Transfer Transferred']['user_id'].unique()
all_new_users['status'] = np.where(all_new_users.index.isin(transferred_users), 'Settled', 'Unsettled')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))
all_new_users.groupby('platform')['status'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax1)
ax1.set_title('Settled vs. Unsettled by Platform', fontsize=14, fontweight='bold')
ax1.set_ylabel('Proportion of Users')
ax1.tick_params(axis='x', rotation=0)

all_new_users.groupby('region')['status'].value_counts(normalize=True).unstack().plot(kind='bar', stacked=True, ax=ax2)
ax2.set_title('Settled vs. Unsettled by Region', fontsize=14, fontweight='bold')
ax2.set_ylabel('Proportion of Users')
ax2.tick_params(axis='x', rotation=0)

plt.suptitle('Profile of Successful vs. Dropped-Off Users', fontsize=18, fontweight='bold')
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig(f'{IMAGE_DIR}/settled_unsettled_distribution.png')
plt.show()

![Settled vs. Unsettled Distribution](images/settled_unsettled_distribution.png)

## 5. Logistic Regression: Key Drivers of Conversion

In [ ]:
model_df = all_new_users.copy()
model_df['converted'] = (model_df['status'] == 'Settled').astype(int)
model_df = pd.get_dummies(model_df, columns=['platform', 'region'], drop_first=True)

X = model_df.drop(columns=['dt', 'experience', 'status', 'converted', 'week', 'event_name'])
y = model_df['converted']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
log_reg = LogisticRegression(class_weight='balanced', solver='liblinear')
log_reg.fit(X_train, y_train)

coefficients = pd.DataFrame({'feature': X.columns, 'coefficient': log_reg.coef_[0]})
coefficients['impact'] = np.exp(coefficients['coefficient'])
coefficients = coefficients.sort_values(by='coefficient', ascending=False)

fig, ax = plt.subplots(figsize=(12, 7))
sns.barplot(x='coefficient', y='feature', data=coefficients, ax=ax, palette=sns.color_palette('Greens_r', n_colors=len(coefficients)))
ax.set_title('Impact of Features on Conversion (Logistic Regression Coefficients)', fontsize=16, fontweight='bold')
ax.set_xlabel('Coefficient (Log-Odds)')
ax.set_ylabel('Feature')
plt.tight_layout()
plt.savefig(f'{IMAGE_DIR}/logistic_regression_coefficients.png')
plt.show()

![Logistic Regression Coefficients](images/logistic_regression_coefficients.png)

## 6. Final Recommendations

### 1. **Fix the Web and Android Experience Immediately.**
   *   **Problem**: Web and Android are the biggest leaks. The logistic regression model confirms that being on Web or Android significantly decreases the odds of converting compared to iOS.
   *   **Action**: Launch a dedicated squad to conduct a full UX/UI audit of the Web and Android onboarding funnels. Use session replay tools to identify specific friction points. Prioritize fixing bugs and simplifying the flow on these platforms.

### 2. **Develop a Non-North American Growth Strategy.**
   *   **Problem**: The product is not resonating outside of North America. The regression shows that being in 'Other' or 'Europe' has a negative impact on conversion compared to North America.
   *   **Action**: Conduct market-specific research in Europe to identify the most trusted and commonly used payment methods. A/B test localization of the user interface, including language and currency presentation.

### 3. **Shift Focus from Acquisition to Onboarding.**
   *   **Problem**: The data clearly shows the problem isn't getting users in the door; it's getting them to complete a transfer.
   *   **Action**: Reallocate resources from top-of-funnel acquisition campaigns to improving the onboarding experience, especially for the identified weak points: Web, Android, and non-NA regions.